# Exercise 13 (2) - Hamiltonian & Lagrangian Neural Networks
### Task
Implement and train a Lagrangian neural network
- Implement the cost computation in the training loop
- Perform a training with the Lagrangian neural network and compare it to the conventional neural network (exercise13_3.ipynb) and the Hamiltonian neural network (`exercise13_1.ipynb`)

### Learning goals
- Familiarize yourself with Hamiltonian dynamics and neural networks

In [ ]:
import torch
import numpy as np
from torch.autograd import grad
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(1)

## User settings

**sampling parameters**

In [ ]:
dx0dt = 1
k = 10
m = 1

tmaxTraining = 1.5
numberOfTrainingSamples = 50
tmaxValidation = 9
numberOfValidationSamples = 300

**neural network architecture**

In [ ]:
neurons = 50
layers = 2

**hyperparameters**

In [ ]:
lr = 1e-3
epochs = 5000

## Utilities

**neural network**

In [ ]:
class FNN(torch.nn.Module):
    def __init__(self, inputSize, outputSize, neurons, layers):
        super().__init__()

        self.layers = layers

        self.linear1 = torch.nn.Linear(inputSize, neurons)
        self.linear2 = torch.nn.ModuleList()
        for i in range(self.layers):
            self.linear2.append(torch.nn.Linear(neurons, neurons))
        self.linear3 = torch.nn.Linear(neurons, outputSize)
        self.activation = torch.nn.Tanh()

    def forward(self, x):
        y = self.activation(self.linear1(x))
        for i in range(self.layers):
            y = self.activation(self.linear2[i](y))
        y = self.linear3(y)
        return y

**derivative function**

In [ ]:
def getDerivative(y, x, n):
    """Compute the nth order derivative of y = f(x) with respect to x."""

    if n == 0:
        return y
    else:
        dy_dx = grad(y, x, torch.ones_like(y), create_graph=True, retain_graph=True)[0]
        return getDerivative(dy_dx, x, n - 1)

## Data generation

**analytical solution**

In [ ]:
omega = np.sqrt(k / m)

A = dx0dt / omega
phi = 0
x = lambda t: A * np.sin(omega * t + phi)
dxdt = lambda t: omega * A * np.cos(omega * t + phi)
ddxdtt = lambda t: - omega ** 2 * A * np.sin(omega * t + phi)

**training data**

In [ ]:
tTraining = np.linspace(0, tmaxTraining, numberOfTrainingSamples)
# TODO add noise
trainingData = torch.vstack((torch.from_numpy(x(tTraining)),
                             torch.from_numpy(dxdt(tTraining)),
                             torch.from_numpy(ddxdtt(tTraining)))).to(torch.float32).T

**validation data**

In [ ]:
tValidation = np.linspace(0, tmaxValidation, numberOfValidationSamples)
validationData = torch.vstack((torch.from_numpy(x(tValidation)),
                               torch.from_numpy(dxdt(tValidation)),
                               torch.from_numpy(ddxdtt(tValidation)))).to(torch.float32).T

**model input**

In [ ]:
modelInputTraining = trainingData[:, :2]
modelInputTraining.requires_grad = True

modelInputValidation = validationData[:, :2]
modelInputValidation.requires_grad = True

## Hamiltonian model training

**pre-processing**

In [ ]:
model = FNN(2, 1, neurons, layers)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
costHistory = np.zeros(epochs)

**training loop**

In [ ]:
for epoch in range(epochs):
    optimizer.zero_grad()

    LPred = model(modelInputTraining)

    # your code goes here: compute the cost function
    
    costHistory[epoch] = cost.detach()

    cost.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print("Epoch: {}/{}\t\tCost function: {:.3E}".format(epoch, epochs, cost.detach()))

## Post-processing

In [ ]:
numberOfTimeSteps = 10000  # for the forward Euler scheme

**forward Euler scheme**

In [ ]:
tPrediction = np.linspace(0, tmaxValidation, numberOfTimeSteps + 1)
dt = tmaxValidation / numberOfTimeSteps
xPrediction = torch.zeros((2, numberOfTimeSteps + 1), requires_grad=False)
xPrediction[:, 0] = validationData[0, :2]

LPredictions = np.zeros(numberOfTimeSteps)
LLabel = 0.5 * m * validationData[:, 1] ** 2 - 0.5 * k * validationData[:, 0] ** 2

for i in range(numberOfTimeSteps):
    currentx = xPrediction[:, i].unsqueeze(0).detach()
    currentx.requires_grad = True
    LPred = model(currentx)
    LPredictions[i] = LPred[0].detach()

    dLPreddx = getDerivative(LPred, currentx, 1)
    Hessian = getDerivative(dLPreddx[:, 0], currentx, 1)[:, 1]
    ddxttPred = (1. / dLPreddx[:, 1]) * (dLPreddx[:, 0] - Hessian * currentx[:, 1])

    xPrediction[:, i + 1] = (xPrediction[:, i] + dt * torch.tensor([currentx[0, 1], ddxttPred[0]])).detach()

**transient response**

In [ ]:
fig, ax = plt.subplots()
ax.plot(tPrediction, xPrediction[0, :], 'k')
ax.plot(tValidation, validationData[:, 0], 'r--')
plt.show()

**position-momentum space**

In [ ]:
fig, ax = plt.subplots()
ax.plot(xPrediction[0, :], xPrediction[1, :], 'k')
ax.plot(validationData[:, 0], validationData[:, 1], 'r--')
plt.show()

**energy evolution**

In [ ]:
fig, ax = plt.subplots()
ax.plot(tPrediction, 0.5 / m * xPrediction[1, :] ** 2, 'k--')
ax.plot(tPrediction, 0.5 * k * xPrediction[0, :] ** 2, 'k:')
ax.plot(tPrediction, 0.5 / m * xPrediction[1, :] ** 2 + 0.5 * k * xPrediction[0, :] ** 2, 'k')

ax.plot(tValidation, 0.5 / m * validationData[:, 1] ** 2, 'r--')
ax.plot(tValidation, 0.5 * k * validationData[:, 0] ** 2, 'r:')
ax.plot(tValidation, 0.5 / m * validationData[:, 1] ** 2 + 0.5 * k * validationData[:, 0] ** 2, 'r')
plt.show()

**Lagrangian prediction**

In [ ]:
fig, ax = plt.subplots()
ax.plot(tPrediction[:-1], LPredictions, 'k')
ax.plot(tValidation, LLabel, 'r--')
plt.show()

**learning history**

In [ ]:
fig, ax = plt.subplots()
ax.plot(costHistory, 'k')
ax.set_yscale('log')
plt.show()